# Introduction to `geoplanar`

### Mexico level 2 - municipalities & US counties

In [ ]:
import geoplanar
import geopandas 

In [ ]:
mex = geopandas.read_file("../geoplanar/datasets/mexico/lvl2/mex_admbnda_adm2_govmex_20210618.shp")
mex.plot() # municipios

In [ ]:
us = geopandas.read_parquet("/Users/ecv/projects/geoplanar/geoplanar/datasets/uscounties.parquet")

In [ ]:
us.plot()

In [ ]:
us = us.to_crs("EPSG:4326")
us.crs

In [ ]:
usmex = us.append(mex)

In [ ]:
usmex.plot()

In [ ]:
usmex.tail()

In [ ]:
usmex.shape

We have appended the Mexico gdf to the US gdf. For now, however, we are going to zoom in the border region to investigate things further:

In [ ]:
from shapely.geometry import box

clipper = geopandas.GeoDataFrame(geometry =[box(-118.2,25.44,-95.17,34.45)])
                                 


In [ ]:
usborder = geopandas.clip(us, clipper) # positional argument matters(see geopandas doc on clipper)

mexborder = geopandas.clip(mex, clipper)

In [ ]:
mexborder.shape

In [ ]:
usborder.shape

In [ ]:
usborder.plot()

In [ ]:
mexborder.plot()

In [ ]:
usmex = usborder.append(mexborder)
usmex.reset_index(inplace=True)

In [ ]:
usmex.plot()

In [ ]:
usmex.shape 

number of geometries checks out

## Border discrepancies

In [ ]:
base = usborder.plot(alpha=0.7, facecolor='none', edgecolor='blue')
_ = mexborder.plot(alpha=0.7, facecolor='none', edgecolor='red', ax=base)
_ = base.set_xlim(-118.2, -95.17)
_ = base.set_ylim(25.44, 34.45)


Notice how some parts of the border have a slight purple color to them because of the overlaps.

## Fixing Overlaps/Overshoots

In [ ]:
import pandas as pd

In [ ]:
usborder['COUNTRY'] = 'US'
usborder.head()

In [ ]:
mexborder['COUNTRY'] = 'MEXICO'
mexborder.head()

In [ ]:
usmex2 = pd.concat([usborder, mexborder])

In [ ]:
usmex2.plot('COUNTRY')

In [ ]:
usmex2.shape

In [ ]:
usmex2 = usmex2.reset_index()

In [ ]:
usmex2.area.sum()  # or use .sum() 

In [ ]:
border_overlaps_removed = geoplanar.trim_overlaps(usmex2) # removing overlaps


In [ ]:
border_overlaps_removed.area.sum() # mexico gets trimmed

In [ ]:
border_overlaps_removed_1 = geoplanar.trim_overlaps(usmex2, largest=False)

In [ ]:
border_overlaps_removed_1.area.sum() # us gets trimmed

In [ ]:
usmex2.explore(tooltip=False) # geopandas explroe function


We can see where the dataframes overlap if we zoom in. The Juarex/El Paso region has multiple instances where
we can see these issues. 
                               

## Fixing undershoots/holes

Trimming the overlaps removes the areas where points belong to both national polygons. What remains after this correction are holes (slivers) where points belong to neither polygon.

In [ ]:
base = border_overlaps_removed.plot()
_ = base.set_xlim(-109.87, -99.14)   
_ = base.set_ylim(27.67, 32.68)

In [ ]:
holes = geoplanar.holes(border_overlaps_removed) # holes left over after removing overlaps

In [ ]:
base = holes.plot(alpha=0.3, facecolor='none', edgecolor='blue')
_ = base.set_xlim(-109.87, -99.14)   
_ = base.set_ylim(27.67, 32.68)

In [ ]:
holes.shape

For the entire border region there are 2191 holes that exist. These can be corrected, by merging the hole with the larger intersecting national polygon:

In [ ]:
final = geoplanar.fill_holes(border_overlaps_removed) # creating final gdf 

In [ ]:
base = final.plot()
_ = base.set_xlim(-109.87, -99.14)   
_ = base.set_ylim(27.67, 32.68)

In [ ]:
final.head()

In [ ]:
h1 = geoplanar.holes(final)

In [ ]:
h1.shape 

We have 0 holes after fixing overlaps, undershoots and filling in the holes.

In [ ]:
base = final.plot(edgecolor='k')
_ = usborder.plot(alpha=0.7, facecolor='none', edgecolor='white', ax = base)
_ = mexborder.plot(alpha=0.7, facecolor='none', edgecolor='red', ax=base)
_ = base.set_xlim(-109.87, -99.14)   
_ = base.set_ylim(27.67, 32.68)



In [ ]:
final.area

In [ ]:
final.shape

In [ ]:
final.head()

After fixing the issues with overlaps, undershots, etc., we can check both gdf to see if we lost any geometries.

In [ ]:
final.plot('COUNTRY')

In [ ]:
usmex.shape

In [ ]:
final.shape

Notice we gained a column because we assigned a 'COUNTRY' label to each border region.

In [ ]:
usmex2.columns.symmetric_difference(usmex.columns)